In [34]:
import numpy as np
import pandas as pd
from linear_regression import LinearRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
def preprocess_data(df):
    df['area_rooms'] = np.sqrt(df.area * df.rooms)
    df['log_price'] = np.log(df['price'])

In [4]:
df = pd.read_csv('prices.txt', dtype=np.float64)
df = df.sample(frac=1).reset_index(drop=True)
prices = df['price']
preprocess_data(df)
df.head()

,area,rooms,price,area_rooms,log_area,log_price
0,1890.0,3.0,329999.0,5670.0,1890.0,12.706845
1,1437.0,3.0,249900.0,4311.0,1437.0,12.428816
2,1203.0,3.0,239500.0,3609.0,1203.0,12.386309
3,1852.0,4.0,299900.0,7408.0,1852.0,12.611204
4,2637.0,3.0,299900.0,7911.0,2637.0,12.611204


In [6]:
plt.figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(2, 3, 1)
plt.xlabel('sqrt area * rooms')
plt.plot(list(np.sqrt(df.area_rooms)), list(df.log_price), 'bo')
plt.subplot(2, 3, 2)
plt.xlabel('rooms')
plt.plot(list(df.rooms), list(df.log_price), 'bo')
plt.subplot(2, 3, 3)
plt.xlabel('area')
plt.plot(list(df.area), list(df.log_price), 'bo')
plt.subplot(2, 3, 4)
plt.xlabel('log area')
plt.plot(list(np.log(df.area)), list(df.log_price), 'bo')
plt.subplot(2, 3, 5)
plt.xlabel('sqrt area')
plt.plot(list(np.sqrt(df.area)), list(df.log_price), 'bo')
plt.show()

In [31]:
def score(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2) ** 0.5

In [80]:
reg = LinearRegression(False, 0)
train_to = 42
X, y = df.loc[:train_to, ['rooms', 'area']].as_matrix(), df.loc[:train_to, ['price']].as_matrix()
Xt, yt = df.loc[train_to:, ['rooms', 'area']].as_matrix(), df.loc[train_to:, ['price']].as_matrix()
maxRooms = float(max(X[:,0]))
maxArea = float(max(X[:, 1]))
maxPrice = max(y)
X[:, 0] = X[:, 0] / maxRooms
X[:, 1] = X[:, 1] / maxArea
y = y / maxPrice
reg.fit(X, y)
reg.weights[0][0] *= maxPrice
reg.weights[0][1] *= maxPrice/maxRooms
reg.weights[0][2] *= maxPrice/maxArea
y = y * maxPrice
print(reg.weights)
Xt = np.c_[np.ones((Xt.shape[0], 1)), Xt]
print('Test score: ', score(np.dot(Xt, reg.weights.T), yt))
fig = plt.figure()
ax = fig.add_subplot(211, projection='3d')
ax.scatter(X[:, 0], X[:, 1], y)
X[:, 0] = X[:, 0] * maxRooms
X[:, 1] = X[:, 1] * maxArea
X = np.c_[np.ones((X.shape[0], 1)), X]
ax2 = fig.add_subplot(212, projection='3d')
ax2.scatter(X[:, 1], X[:, 2], np.dot(X, reg.weights.T))
plt.show()

[[ 87669.16545727  -3665.52997963    131.4952628 ]]
Test score:  58804.211438


In [22]:
reg2 = LinearRegression(False, 0, 'exact')
train_to = 40
X, y = df.loc[:train_to, ['rooms', 'area']].as_matrix(), df.loc[:train_to, ['price']].as_matrix()
Xt, yt = df.loc[train_to:, ['rooms', 'area']].as_matrix(), df.loc[train_to:, ['price']].as_matrix()
reg2.fit(X, y)
X = np.c_[np.ones((X.shape[0], 1)), X]
Xt = np.c_[np.ones((Xt.shape[0], 1)), Xt]
print(reg2.weights)
print('Test score', score(np.dot(X, reg2.weights.T), y))
print('Train score', score(np.dot(Xt, reg2.weights.T), yt))

[[ 92301.89160212  -3666.5637514     130.37743339]]
64887.4946275
72683.7649132


In [78]:
X[1]

IndexError: list index out of range